# Ray

## OVERVIEW OF RAY

### Installing Ray

#### Official Releases

You can install the latest official version of Ray as follows. Official releases are produced according to the release process doc.
```bash
# Install pip
sudo apt update
sudo apt install python3-pip
# Install ray
pip3 install -U ray
```

## RAY CLUSTERS/AUTOSCALER

### Ray Cluster Quick Start
[Ray Cluster Quick Start](https://docs.ray.io/en/master/cluster/quickstart.html#)

#### Install some Python dependencies

Before we start, you will need to install some Python dependencies as follows:
```bash
pip3 install -U 'ray[default]'
```

#### Create a (basic) Python application

We will write a simple Python application that tracks the IP addresses of the machines that its tasks are executed on:\
我们将编写一个简单的Python应用程序，跟踪执行其任务的机器的IP地址:
```python
from collections import Counter
import socket
import time

import ray

ray.init()

print('''This cluster consists of
    {} nodes in total
    {} CPU resources in total
'''.format(len(ray.nodes()), ray.cluster_resources()['CPU']))

@ray.remote
def f():
    time.sleep(0.001)
    # Return IP address.
    return socket.gethostbyname(socket.gethostname())

object_ids = [f.remote() for _ in range(10000)]
ip_addresses = ray.get(object_ids)

print('Tasks executed')
for ip_address, num_tasks in Counter(ip_addresses).items():
    print('    {} tasks on {}'.format(num_tasks, ip_address))
```

Save this application as `script.py`:
```bash
vim script.py
```

Execute it by running the command:
```bash
python3 script.py
```

Should now output something like:
```bash
This cluster consists of
    1 nodes in total
    4.0 CPU resources in total

Tasks executed
    10000 tasks on 127.0.0.1
```

#### Launch a cluster

After defining our configuration, we will use the Ray Cluster Launcher to start a cluster on the cloud, creating a designated “head node” and worker nodes. To start the Ray cluster, we will use the Ray CLI. Run the following command:\
在定义了我们的配置之后，我们将使用Ray Cluster Launcher在云上启动一个集群，创建一个指定的“头节点”和工作节点。为了启动Ray集群，我们将使用Ray CLI。执行如下命令:
```bash
ray up -y tune-default.yaml
```

#### Run the application in the cloud

We are now ready to execute the application in across multiple machines on our Ray cloud cluster. First, we need to edit the initialization command `ray.init()` in `script.py`.\
我们现在可以在Ray云集群的多台机器上执行应用程序了。首先，我们需要编辑`script.py`中的初始化命令`ray.init()`。
```bash
vim script.py
```
Change it to\
将其更改为
```python
ray.init(address='auto')
# Pass `include_webui=True` to access the dashboard
ray.init(address='auto', include_webui=True)
```
This will allow Ray to connect to the remote cluster.\
这将允许Ray连接到远程集群。

Next, run the following command:
```bash
ray submit tune-default.yaml script.py
ray submit tune-default.yaml script.py -- --ray-address=192.168.2.238:6379
```

The output should now look similar to the following:
```bash
This cluster consists of
    3 nodes in total
    6.0 CPU resources in total

Tasks executed
    3425 tasks on xxx.xxx.xxx.xxx
    3834 tasks on xxx.xxx.xxx.xxx
    2741 tasks on xxx.xxx.xxx.xxx
```

### Launching Cloud Clusters
[Launching Cloud Clusters](https://docs.ray.io/en/master/cluster/cloud.html)

#### Local On Premise Cluster (List of nodes)

You would use this mode if you want to run distributed Ray applications on some local nodes available on premise.\
如果您想在某些本地节点上运行分布式Ray应用程序，您可以使用这种模式。

The most preferable way to run a Ray cluster on a private cluster of hosts is via the Ray Cluster Launcher.\
在主机的私有集群上运行Ray集群的最佳方式是通过Ray cluster Launcher。

There are two ways of running private clusters:\
有两种方式运行私有集群:

* Manually managed, i.e., the user explicitly specifies the head and worker ips.\
手动管理，即用户显式指定头ip和工人ip。
* Automatically managed, i.e., the user only specifies a coordinator address to a coordinating server that automatically coordinates its head and worker ips.\
自动管理，即，用户只指定一个协调服务器的协调地址，自动协调它的头和工作ip。

___
**Tip**

To avoid getting the password prompt when running private clusters make sure to setup your ssh keys on the private cluster as follows:\
为了避免在运行私有集群时出现密码提示，请确保在私有集群上设置ssh密钥如下：
```bash
$ ssh-keygen
$ cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
```
___

You can get started by filling out the fields in the provided [ray/python/ray/autoscaler/local/example-full.yaml](https://github.com/ray-project/ray/tree/master/python/ray/autoscaler/local/example-full.yaml). Be sure to specify the proper `head_ip`, list of `worker_ips`, and the `ssh_user` field.\
您可以通过填写提供的[ray/python/ray/autoscaler/local/example-full.yaml](https://github.com/ray-project/ray/tree/master/python/ray/autoscaler/local/example-full.yaml)中的字段开始。请确保指定正确的`head_ip`、`worker_ips`列表和`ssh_user`字段。
```bash
vim ray/python/ray/autoscaler/local/example-full.yaml
```

```yaml
# An unique identifier for the head node and workers of this cluster.
cluster_name: default

## NOTE: Typically for local clusters, min_workers == max_workers == len(worker_ips).

# The minimum number of workers nodes to launch in addition to the head
# node. This number should be >= 0.
# Typically, min_workers == max_workers == len(worker_ips).
min_workers: 3

# The maximum number of workers nodes to launch in addition to the head node.
# This takes precedence over min_workers.
# Typically, min_workers == max_workers == len(worker_ips).
max_workers: 3

# The autoscaler will scale up the cluster faster with higher upscaling speed.
# E.g., if the task requires adding more nodes then autoscaler will gradually
# scale up the cluster in chunks of upscaling_speed*currently_running_nodes.
# This number should be > 0.
upscaling_speed: 1.0

idle_timeout_minutes: 5

# Local specific configuration.
provider:
    type: local
    head_ip: 192.168.2.238
    worker_ips: [192.168.2.17, 192.168.2.27]
    # Optional when running automatic cluster management on prem. If you use a coordinator server,
    # then you can launch multiple autoscaling clusters on the same set of machines, and the coordinator
    # will assign individual nodes to clusters as needed.
    #    coordinator_address: "<host>:<port>"

# How Ray will authenticate with newly launched nodes.
auth:
    ssh_user: ubuntu
    # Optional if an ssh private key is necessary to ssh to the cluster.
    ssh_private_key: ~/.ssh/id_rsa

# Leave this empty.
head_node: {}

# Leave this empty.
worker_nodes: {}

# Files or directories to copy to the head and worker nodes. The format is a
# dictionary from REMOTE_PATH: LOCAL_PATH, e.g.
file_mounts: {
#    "/path1/on/remote/machine": "/path1/on/local/machine",
#    "/path2/on/remote/machine": "/path2/on/local/machine",
}

# Files or directories to copy from the head node to the worker nodes. The format is a
# list of paths. The same path on the head node will be copied to the worker node.
# This behavior is a subset of the file_mounts behavior. In the vast majority of cases
# you should just use file_mounts. Only use this if you know what you're doing!
cluster_synced_files: []

# Whether changes to directories in file_mounts or cluster_synced_files in the head node
# should sync to the worker node continuously
file_mounts_sync_continuously: False

# Patterns for files to exclude when running rsync up or rsync down
rsync_exclude:
    - "**/.git"
    - "**/.git/**"

# Pattern files to use for filtering out files when running rsync up or rsync down. The file is searched for
# in the source directory and recursively through all subdirectories. For example, if .gitignore is provided
# as a value, the behavior will match git's behavior for finding and using .gitignore files.
rsync_filter:
    - ".gitignore"

# List of commands that will be run before `setup_commands`. If docker is
# enabled, these commands will run outside the container and before docker
# is setup.
initialization_commands: []

# List of shell commands to run to set up each nodes.
setup_commands: []
    # Note: if you're developing Ray, you probably want to create a Docker image that
    # has your Ray repo pre-cloned. Then, you can replace the pip installs
    # below with a git checkout <your_sha> (and possibly a recompile).
    # To run the nightly version of ray (as opposed to the latest), either use a rayproject docker image
    # that has the "nightly" (e.g. "rayproject/ray-ml:nightly-gpu") or uncomment the following line:
    # - pip install -U "ray[default] @ https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-2.0.0.dev0-cp37-cp37m-manylinux2014_x86_64.whl"

# Custom commands that will be run on the head node after common setup.
head_setup_commands: []

# Custom commands that will be run on worker nodes after common setup.
worker_setup_commands: []

# Command to start ray on the head node. You don't need to change this.
head_start_ray_commands:
    - ray stop
    - ulimit -c unlimited && ray start --head --port=6379 --autoscaling-config=~/ray_bootstrap_config.yaml

# Command to start ray on worker nodes. You don't need to change this.
worker_start_ray_commands:
    - ray stop
    - ray start --address=$RAY_HEAD_IP:6379
```

Test that it works by running the following commands from your local machine:\
通过在本地机器上运行以下命令来测试它的工作：
```bash
# Create or update the cluster. 
# When the command finishes, 
# it will print out the command that can be used to get a remote shell into the head node.
$ ray up ray/python/ray/autoscaler/local/example-full.yaml

# Get a remote screen on the head node.
$ ray attach ray/python/ray/autoscaler/local/example-full.yaml
# Try running a Ray program with 'ray.init(address="auto")'.

# Tear down the cluster
$ ray down ray/python/ray/autoscaler/local/example-full.yaml
```

#### Manual Ray Cluster Setup

The most preferable way to run a Ray cluster is via the Ray Cluster Launcher. However, it is also possible to start a Ray cluster by hand.\
运行射线集群的最佳方式是通过射线集群启动器。然而，也可以手动启动一个Ray集群。

This section assumes that you have a list of machines and that the nodes in the cluster can communicate with each other. It also assumes that Ray is installed on each machine.\
本节假设您有一个机器列表，并且集群中的节点可以彼此通信。它还假设在每台机器上都安装了Ray。

##### Starting Ray on each machine

On the head node (just choose some node to be the head node), run the following. If the `--port` argument is omitted, Ray will choose port 6379, falling back to a random port.\
在头节点上(只需选择某个节点作为头节点)，运行以下命令。如果省略了`--port`参数，Ray将选择端口6379，返回到一个随机端口。
```bash
ray start --head
```
You should see somthing like:
```bash
Local node IP: 192.168.2.238
2021-05-09 20:53:07,861	INFO services.py:1262 -- View the Ray dashboard at http://127.0.0.1:8265

--------------------
Ray runtime started.
--------------------

Next steps
  To connect to this Ray runtime from another node, run
    ray start --address='192.168.2.238:6379' --redis-password='5241590000000000'
  
  Alternatively, use the following Python code:
    import ray
    ray.init(address='auto', _redis_password='5241590000000000')
  
  If connection fails, check your firewall settings and network configuration.
  
  To terminate the Ray runtime, run
    ray stop
```

The command will print out the address of the Redis server that was started (the local node IP address plus the port number you specified).\
该命令将打印出启动的Redis服务器的地址(本地节点IP地址加上您指定的端口号)。


**Then on each of the other nodes**, run the following. Make sure to replace `<address>` with the value printed by the command on the head node (it should look something like `123.45.67.89:6379`).\
**然后在每个其他节点上**，运行以下命令。确保将`<address>`替换为该命令在头节点上打印的值(它应该类似于`123.45.67.89:6379`)。
```bash
ray start --address=<address> --redis-password='<password>'
ray start --address='192.168.2.238:6379' --redis-password='5241590000000000'
```
You should see something like this:
```bash
Local node IP: 192.168.2.238

--------------------
Ray runtime started.
--------------------

To terminate the Ray runtime, run
  ray stop
```

If you wish to specify that a machine has 10 CPUs and 1 GPU, you can do this with the flags `--num-cpus=10` and `--num-gpus=1`.\
如果你想指定一台机器有10个cpu和1个GPU，你可以使用标记`--num-cpus=10`和`--num-gpus=1`来完成。

If you see `Ray runtime started`, then the node successfully connected to the IP address at the `--port`. You should now be able to connect to the cluster with `ray.init(address='auto')`.\
如果你看到`Ray runtime started`，则节点成功连接到`--port`的IP地址。您现在应该能够使用`ray.init(address='auto')`连接到集群。

Next, run the following command to test:
```bash
python3 script.py
```
The output should now look similar to the following:
```bash
2021-05-09 20:37:04,489	INFO worker.py:663 -- Connecting to existing Ray cluster at address: 192.168.2.238:6379
This cluster consists of
    3 nodes in total
    3.0 CPU resources in total

Tasks executed
    3475 tasks on 192.168.2.238
    3646 tasks on 192.168.2.27
    2879 tasks on 192.168.2.17
```


## RAY TUNE

To run this example, install the following:
```bash
pip3 install 'ray[tune]'
```

### Tune Distributed Experiments
[Tune Distributed Experiments](https://docs.ray.io/en/master/tune/tutorials/tune-distributed.html)

#### Local Cluster Setup

If you already have a list of nodes, you can follow the local private cluster setup. Below is an example cluster configuration as `tune-default.yaml`:\
如果您已经有一个节点列表，那么您可以按照本地私有集群设置。下面是`tune-default.yaml`集群配置示例：
```yaml
cluster_name: local-default
provider:
    type: local
    head_ip: YOUR_HEAD_NODE_HOSTNAME
    worker_ips: [WORKER_NODE_1_HOSTNAME, WORKER_NODE_2_HOSTNAME, ... ]
auth: {ssh_user: YOUR_USERNAME, ssh_private_key: ~/.ssh/id_rsa}
## Typically for local clusters, min_workers == max_workers.
min_workers: 3
max_workers: 3
setup_commands:  # Set up each node.
    - pip install ray torch torchvision tabulate tensorboard
```

Create a `tune-default.yaml`:
```bash
sudo vim tune-default.yaml
```
with the example configuration:
```yaml
cluster_name: local-default
provider:
    type: local
    head_ip: mengfeiliang-a3-m
    worker_ips: [mengfeiliang-a3-w1, mengfeiliang-a3-w2]
    cache_stopped_nodes: False
auth: {ssh_user: ubuntu, ssh_private_key: ~/.ssh/id_rsa}
## Typically for local clusters, min_workers == max_workers.
min_workers: 3
max_workers: 3
setup_commands:  # Set up each node.
    - pip3 install ray torch torchvision tabulate tensorboard
```

`ray up` starts Ray on the cluster of nodes.\
`ray up`在节点集群上启动ray。
```bash
ray up tune-default.yaml
```

`ray submit` uploads `tune_script.py` to the cluster and runs `python tune_script.py [args]`.\
`ray submit`上传`tune_script.py`到集群并运行`python tune_script.py [args]`。
```bash
ray submit tune-default.yaml tune_script.py -- --ray-address=localhost:6379
```

### Tune API Reference

#### Scikit-Learn API (tune.sklearn)

[Scikit-Learn API (tune.sklearn)](https://docs.ray.io/en/master/tune/api_docs/sklearn.html)

```bash
python3 tune_script.py
```

## DEVELOPMENT AND RAY INTERNALS

### Building Ray from Source
[Building Ray from Source](https://docs.ray.io/en/master/development.html#building-ray-python-only)

#### Building Ray (Python Only)

1. Pip install the latest Ray wheels.
```bash
pip3 install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-2.0.0.dev0-cp38-cp38-manylinux2014_x86_64.whl
```

2. Fork and clone the project to your machine. Connect your repository to the upstream (main project) ray repository.
```bash
git clone https://github.com/[your username]/ray.git
cd ray
git remote add upstream https://github.com/ray-project/ray.git
# Make sure you are up-to-date on master.
```

3. Replace Python files in the installed package with your local editable copy. We provide a simple script to help you do this: `python python/ray/setup-dev.py`. Running the script will remove the `ray/tune`, `ray/rllib`, `ray/autoscaler` dir (among other directories) bundled with the `ray` pip package, and replace them with links to your local code. This way, changing files in your git clone will directly affect the behavior of your installed ray.\
用本地可编辑副本替换安装包中的Python文件。我们提供了一个简单的脚本来帮助你做到这一点：`python python/ray/setup-dev.py`。运行该脚本将删除与`ray` pip包绑定的`ray/tune`，`ray/rllib`，`ray/autoscaler`目录(以及其他目录)，并将它们替换为指向本地代码的链接。这样，改变文件在你的git克隆将直接影响你安装的ray的行为。
___
**Warning**

Do not run `pip uninstall ray` or `pip install -U` (for Ray or Ray wheels) if setting up your environment this way. To uninstall or upgrade, you must first `rm -rf` the pip-installation site (usually a site-packages/ray location), then do a pip reinstall (see 1. above), and finally run the above `setup-dev.py` script again.\
如果以这种方式设置环境，请不要运行`pip uninstall ray`或`pip install -U`(用于ray或ray wheels)。要卸载或升级，您必须首先`rm -rf` pip安装位置(通常是`site-packages/ray`位置)，然后进行pip重新安装(参见1。最后再次运行上面的`setup-dev.py`脚本。

```bash
rm -rf /home/ubuntu/.local/lib/python3.8/site-packages/ray
```
___

```bash
git clone https://github.com/ray-project/ray.git
cd ray
python3 python/ray/setup-dev.py
# This replaces miniconda3/lib/python3.7/site-packages/ray/tune
# with your local `ray/python/ray/tune`.
```

#### Building Ray on Linux (full)

To build Ray, first install the following dependencies.

For Ubuntu, run the following commands:
```bash
sudo apt-get update
sudo apt-get install -y build-essential curl unzip psmisc

pip3 install cython==0.29.0 pytest
```

Ray can be built from the repository as follows.
```bash
git clone https://github.com/ray-project/ray.git

# Install Bazel.
# (Windows users: please manually place Bazel in your PATH, and point BAZEL_SH to MSYS2's Bash.)
ray/ci/travis/install-bazel.sh

# (requires Node.js, see https://nodejs.org/ for more information).
sudo apt-get install nodejs
sudo apt-get install npm
# Build the dashboard
pushd ray/dashboard/client
npm install
npm run build
popd

# Install Ray.
cd ray/python
pip3 install -e . --verbose  # Add --user if you see a permission denied error.
pip3 install -e . --verbose --user
```

The `-e` means “editable”, so changes you make to files in the Ray directory will take effect without reinstalling the package.

___
**Warning**

if you run `python setup.py install`, files will be copied from the Ray directory to a directory of Python packages (`/lib/python3.6/site-packages/ray`). This means that changes you make to files in the Ray directory will not have any effect.\
如果运行`python setup.py install`，文件将从Ray目录复制到包含python包的目录(`/lib/python3.6/site-packages/Ray`)。这意味着您对Ray目录中的文件所做的更改不会有任何效果。
___

